In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2
scikit-learn-intelex==2021.20210714.120553


In [1]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [3]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)
y_pred

array([14.53986486, 13.74042222, 15.59333908, ..., 15.83492293,
       16.78317605, 19.65462607])

In [6]:
y_pred.mean()

16.191691679979066

In [7]:
year = 2021
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [9]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred
df_result

,ride_id,predicted_duration
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206
...,...,...
1037687,2021/02_1037687,12.433246
1037688,2021/02_1037688,12.910885
1037689,2021/02_1037689,15.834923
1037690,2021/02_1037690,16.783176


In [ ]:
df_result.to_parquet(
    'df_result.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)